In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output, JSON
import ipywidgets as widgets
import json

In [7]:
def patient_info_visualization(patient_id, folder_path='./pads-dataset-1.0.0/patients/'):
    patient_id_str = f"{patient_id:03d}"
    
    json_path = os.path.join(folder_path, f"patient_{patient_id_str}.json")
    
    if not os.path.exists(json_path):
        print(f"File not found: {json_path}")
        return
        
    with open(json_path, 'r') as f:
        patient_data = json.load(f)
    
    print(f"Patient {patient_id_str} Data Loaded:")
    
    patient_df = pd.json_normalize(patient_data)
    display(patient_df)

In [8]:
def patient_quest_visualization(patient_id, folder_path='./pads-project-main/data/questionnaire/'):
    
    
    ques_bin_file = os.path.join(folder_path, f"{patient_id:03d}_ml.bin")
    
    ques_data = np.fromfile(ques_bin_file, dtype=np.float32)

    questions_info = [
        # Category, Question Number, Symptom
        ("Gastrointestinal tract", 1, "Dribbling"),
        ("Gastrointestinal tract", 3, "Swallowing"),
        ("Gastrointestinal tract", 4, "Vomiting"),
        ("Gastrointestinal tract", 5, "Constipation"),
        ("Gastrointestinal tract", 6, "Bowel inconsistence"),
        ("Gastrointestinal tract", 7, "Bowel emptying incomplete"),
        ("Urinal tract", 8, "Urgency"),
        ("Urinal tract", 9, "Nocturia"),
        ("Pain", 10, "Pains"),
        ("Miscellaneous", 11, "Weight"),
        ("Miscellaneous", 28, "Sweating"),
        ("Miscellaneous", 29, "Diplopia"),
        ("Apathy/attention/memory", 12, "Remembering"),
        ("Apathy/attention/memory", 13, "Loss of interest"),
        ("Apathy/attention/memory", 15, "Concentrating"),
        ("Distortion of perception", 2, "Taste/smelling"),
        ("Distortion of perception", 14, "Hallucinations"),
        ("Distortion of perception", 30, "Delusions"),
        ("Depression/anxiety", 16, "Sad, blues"),
        ("Depression/anxiety", 17, "Anxiety"),
        ("Sexual function", 18, "Sex drive"),
        ("Sexual function", 19, "Sex difficulty"),
        ("Cardiovascular", 20, "Dizzy"),
        ("Cardiovascular", 21, "Falling"),
        ("Cardiovascular", 27, "Swelling"),
        ("Sleep/fatigue", 22, "Daytime sleepiness"),
        ("Sleep/fatigue", 23, "Insomnia"),
        ("Sleep/fatigue", 24, "Intense vivid dreams"),
        ("Sleep/fatigue", 25, "Acting out during dreams"),
        ("Sleep/fatigue", 26, "Restless legs"),
    ]


    data = []
    for cat, q_num, symptom in questions_info:
        
        answer_idx = q_num - 1
        if answer_idx >= len(ques_data):
            answer = "N/A"
        else:
            if ques_data[answer_idx] == 1:
                answer = '<span style="color: green; font-weight: bold;">YES</span>'
            else:
                answer = '<span style="color: red; font-weight: bold;">NO</span>'
        data.append([cat, symptom, q_num, answer])

    df = pd.DataFrame(data, columns=["Category", "Symptom", "Question Number", "Answer"])

    display(HTML(df.to_html(escape=False, index=False)))

In [9]:
def patient_mov_visualization(patient_id, folder_path='./pads-project-main/data/movements/'):
    move_bin_file = f"./pads-project-main/data/movement/{patient_id:03d}_ml.bin"
    
    move_data = np.fromfile(move_bin_file, dtype=np.float32)
    
    num_sensors = 6  # Accel_X, Accel_Y, Accel_Z, Gyro_X, Gyro_Y, Gyro_Z
    num_samples = move_data.shape[0] // num_sensors
    sensor_data = move_data.reshape((num_samples, num_sensors))
    
    sampling_rate = 100  # 100 Hz → 100 samples per second

    task_durations = {
        "Relaxed1": (0, 976),  
        "Relaxed2": (976, 2 * 976),
        "RelaxedTask1": (2 * 976 , 3 * 976),
        "RelaxedTask2": (3 * 976 , 4 * 976),
        "StretchHold": (4 * 976 , 5 * 976),
        "HoldWeight": (5 * 976 , 6 * 976),
        "DrinkGlas": (6 * 976 , 7 * 976),
        "CrossArms": (7 * 976 , 8 * 976),
        "TouchNose": (8 * 976 , 9 * 976),
        "Entrainment1": (9 * 976 , 10 * 976),
        "Entrainment2": (10 * 976 , 11 * 976)
    }

    fig, axes = plt.subplots(len(task_durations), 2, figsize=(12, len(task_durations) * 2))

    for idx, (task, (start_idx, end_idx)) in enumerate(task_durations.items()):

        task_time = np.arange((end_idx - start_idx)) / sampling_rate
        task_data = sensor_data[start_idx:end_idx, :]

        axes[idx, 0].plot(task_time, task_data[:, 0], label="Accel X")
        axes[idx, 0].plot(task_time, task_data[:, 1], label="Accel Y")
        axes[idx, 0].plot(task_time, task_data[:, 2], label="Accel Z")
        axes[idx, 0].set_title(f"PID: {patient_id:03d}, {task} - Accelerometer")
        axes[idx, 0].set_xlabel("Time (s)")
        axes[idx, 0].set_ylabel("Acceleration (m/s²)")
        axes[idx, 0].legend()

        axes[idx, 1].plot(task_time, task_data[:, 3], label="Gyro X")
        axes[idx, 1].plot(task_time, task_data[:, 4], label="Gyro Y")
        axes[idx, 1].plot(task_time, task_data[:, 5], label="Gyro Z")
        axes[idx, 1].set_title(f"PID: {patient_id:03d}, {task} - Gyroscope")
        axes[idx, 1].set_xlabel("Time (s)")
        axes[idx, 1].set_ylabel("Rotation (°/s)")
        axes[idx, 1].legend()

    plt.tight_layout()
    plt.show()
    

In [10]:
participant_id = widgets.Text(
    value='1',
    description='Participant ID (1 ~ 469):',
    style={'description_width': 'initial'}
)
plot_button = widgets.Button(description="Plot Data", button_style='info')

output = widgets.Output()

def plot_data(b):
    with output:
        clear_output(wait=True)  # Clear previous output

        # Load movement data
        patient_id = int(participant_id.value)
        patient_info_visualization(patient_id)
        patient_quest_visualization(patient_id)
        patient_mov_visualization(patient_id)


plot_button.on_click(plot_data)


display(participant_id)

display(plot_button)
display(output)


Text(value='1', description='Participant ID (1 ~ 469):', style=TextStyle(description_width='initial'))

Button(button_style='info', description='Plot Data', style=ButtonStyle())

Output()

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

participant_id = widgets.Text(
    value='55',
    description='Participant ID (1 ~ 469):',
    style={'description_width': 'initial'}
)

sensor_options = [
    ('Accel X', 0), ('Accel Y', 1), ('Accel Z', 2),
    ('Gyro X', 3), ('Gyro Y', 4), ('Gyro Z', 5)
]
sensor_checkboxes = {label: widgets.Checkbox(value=False, description=label) for label, _ in sensor_options}

questionnaire_dropdown = widgets.Dropdown(
    options=["Show All"] + [f"Question {i+1}" for i in range(30)],  # Assume 30 questions
    description="Questionnaire:",
    style={'description_width': 'initial'}
)

plot_button = widgets.Button(description="Plot Data", button_style='info')

# Output widget to display results
output = widgets.Output()

# Function to load and visualize the data
def plot_data(b):
    with output:
        clear_output(wait=True)  # Clear previous output

        # Load movement data
        p_id_int = int(participant_id.value)
        move_bin_file = f"./pads-project-main/data/movement/{p_id_int:03d}_ml.bin"
        print(f"Movement file Path: ./pads-project-main/data/questionnaire/{p_id_int:03d}_ml.bin")
        try:
            move_data = np.fromfile(move_bin_file, dtype=np.float32)
            num_sensors = 6  # Accel X, Accel Y, Accel Z, Gyro X, Gyro Y, Gyro Z
            num_samples = move_data.shape[0] // num_sensors
            move_data = move_data.reshape((num_samples, num_sensors))
            time_axis = np.arange(num_samples) / 100  # 100 Hz sampling rate
        except:
            print(f"Error loading movement file: {move_bin_file}")
            return

        # Extract selected sensor data
        selected_sensors = [i for label, i in sensor_options if sensor_checkboxes[label].value]
        if not selected_sensors:
            print("No sensor selected for visualization.")
            return

        # Plot selected sensor data
        plt.figure(figsize=(12, 5))
        for i in selected_sensors:
            plt.plot(time_axis, move_data[:, i], label=sensor_options[i][0])

        plt.title(f"Movement Data for Participant {participant_id.value}")
        plt.xlabel("Time (s)")
        plt.ylabel("Sensor Values")
        plt.legend()
        plt.grid()
        plt.show()

        # Load questionnaire data
        questionnaire_file = f"./pads-project-main/data/questionnaire/{p_id_int:03d}_ml.bin"
        print(f"Questionnarrie File Path: ./pads-project-main/data/questionnaire/{p_id_int:03d}_ml.bin")
        try:
            questionnaire_data = np.fromfile(questionnaire_file, dtype=np.float32)
            print("Questionnaire data shape: ", questionnaire_data.shape)
            if questionnaire_dropdown.value == "Show All":
                print(f"Questionnaire Responses for Participant {participant_id.value}:")
                print(questionnaire_data)
            else:
                question_idx = int(questionnaire_dropdown.value.split(" ")[1]) - 1
                print(f"Response to {questionnaire_dropdown.value}: {questionnaire_data[question_idx]}")
        except:
            print(f"Error loading questionnaire file: {questionnaire_file}")

# Link button click event to function
plot_button.on_click(plot_data)

# Display UI elements
display(participant_id)
display(widgets.VBox(list(sensor_checkboxes.values())))
display(questionnaire_dropdown)
display(plot_button)
display(output)


Text(value='55', description='Participant ID (1 ~ 469):', style=TextStyle(description_width='initial'))

Dropdown(description='Questionnaire:', options=('Show All', 'Question 1', 'Question 2', 'Question 3', 'Questio…

Button(button_style='info', description='Plot Data', style=ButtonStyle())

Output()

In [9]:
import sys
print(sys.version)
print(sys.version_info)

packages = {
    "torch": "torch",
    "torchvision": "torchvision",
    "torchaudio": "torchaudio",
    "tensorflow": "tensorflow",
    "keras": "keras",
    "numpy": "numpy",
    "pandas": "pandas",
    "scikit-learn": "sklearn",
    "scipy": "scipy",
    "matplotlib": "matplotlib",
    "seaborn": "seaborn",
    "fastai": "fastai",
    "tsai": "tsai",
    "catboost": "catboost"
}

print("\nInstalled Package Versions:\n")

for name, module_name in packages.items():
    try:
        module = __import__(module_name)
        version = getattr(module, '__version__', 'Unknown')
        print(f"{name:<15}: {version}")
    except ImportError:
        print(f"{name:<15}: ❌ Not installed")

3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) 
[GCC 12.3.0]
sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)

Installed Package Versions:

torch          : 2.4.1
torchvision    : 0.19.1
torchaudio     : 2.4.1
tensorflow     : 2.14.0
keras          : 2.14.0
numpy          : 1.26.4
pandas         : 2.2.3
scikit-learn   : 1.6.1
scipy          : 1.13.1
matplotlib     : 3.9.4
seaborn        : 0.13.2
fastai         : 2.7.19
tsai           : 0.4.0
catboost       : 1.2.7
